In [ ]:
# %%
import pandas as pd
import re
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import requests
import time
from openpyxl import load_workbook
from deep_translator import GoogleTranslator
import numpy as np

# Get the current date and time
now = datetime.now()

#define system path start
# change here
path = r'C:\Users\TaYu430\OneDrive - HP Inc\General - Core Team Laser & Ink\For Lip Kiat and Choon Chong\Web review\14_Text_mining\Tassel\\'
excel_path = r'C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\\'

#define system path start

# Format the timestamp as a string
timestamp = now.strftime("%Y-%m-%d %H:%M:%S")

# Print the timestamp
print("Current Timestamp:", timestamp)

print('Running Walmart Script')

# %%


excel_file_path = excel_path + "Star rating scrape URL and info - NPI.xlsx"
sheet_name = "data_new"

# Read the Excel sheet into a DataFrame
df_amazon = pd.read_excel(excel_file_path, sheet_name=sheet_name, engine='openpyxl')
df_amazon['HP Model Number'] = df_amazon['HP Model Number'].astype(str)
df_amazon['Comp Model number'] = df_amazon['Comp Model number'].fillna(0).round(0).astype(int).astype(str)
df_amazon

# %%
excel_file_path = excel_path + "Star rating scrape URL and info - NPI.xlsx"
sheets = 'review_template'
review_template = pd.read_excel(excel_file_path, sheet_name=sheets, engine='openpyxl')
review_template


# Walmart
def get_soup_walmart(url):
    header = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://www.walmart.com/',  # Referer header might be required for some websites
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        'Downlink': '10',
        'Dpr': '1',
        'Sec-Ch-Ua': '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1'
    }
    sheets = "api"
    api = pd.read_excel(excel_file_path, sheet_name=sheets)
    api_key = api['API'][0]
    
    api = f"https://api.scrapingdog.com/scrape?api_key={api_key}&url={url}"
    response = requests.get("https://api.scrapingdog.com/scrape", params={
        'api_key': api_key,
        'url': url,
        'dynamic': 'true',
    })
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


def get_page_number(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://www.walmart.com/',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        'Sec-Ch-Ua': '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1'
    }

    # Load API key from Excel
    sheets = "api"
    api = pd.read_excel(excel_file_path, sheet_name=sheets)  # Use global variable here
    api_key = api['API'][0]
    # Scrapingdog API with JavaScript rendering enabled
    response = requests.get("https://api.scrapingdog.com/scrape", params={
        'api_key': api_key,
        'url': url,
        'dynamic': 'true',
    })
    
    # Ensure the response is successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')        
        # Finding the page number elements with class or attribute related to pagination
        page_number_elements = soup.find_all(
            lambda tag: tag.name == 'a' and 'page-number' in tag.get('data-automation-id', '')
        )
        
        # Extracting the page numbers
        page_numbers = [int(element.text) for element in page_number_elements]

        if page_numbers:
            last_page_number = max(page_numbers)
            return last_page_number
        else:
            print("No page numbers found. Assuming only one page.")
            return 1
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None



def get_review_walmart(url):
    extracted_reviews = []
    retry_count = 0
    sheets = "api"
    api = pd.read_excel(excel_file_path, sheet_name=sheets)
    api_key = api['API'][0]

    try:
        response = requests.get("https://api.scrapingdog.com/scrape", params={
            'api_key': api_key,
            'url': url,
            'dynamic': 'true',
        })
        soup = BeautifulSoup(response.text, 'html.parser')

        li_elements = soup.find_all('div', class_=re.compile(r'overflow-visible b--none mt\d-l ma0 dark-gray'))
        title_all = soup.find('a', class_='w_x7ug f6 dark-gray')
        if title_all:
            title = title_all.get('href')
            pattern = r'(\d{4}[a-zA-Z]?)-'
            model = re.findall(pattern, title)
            li_elements = soup.find_all('div', class_=re.compile(r'overflow-visible b--none mt\d-l ma0 dark-gray'))
            
            for li_tag in li_elements:
                product = {}
                product['Model'] = title
                # Extracting the review rating
                review_rating_element = li_tag.select_one('.w_iUH7')
                product['Review rating'] = review_rating_element.text if review_rating_element else None
    
                # Checking if it's a verified purchase
                verified_purchase_element = li_tag.select_one('.pl2.green.b.f7.self-center')
                product['Verified Purchase or not'] = verified_purchase_element.text if verified_purchase_element else None
    
                # Extracting the review date
                review_date_element = li_tag.select_one('.f7.gray')
                date = review_date_element.text if review_date_element else None
                data = pd.to_datetime(date) - timedelta(days=1)
                formatted_data = data.strftime('%m/%d/%Y')
                product['Review date'] = formatted_data
                
                # Extracting the review title
                review_title_element = li_tag.select_one('.w_kV33.w_Sl3f.w_mvVb.f5.b')
                product['Review title'] = review_title_element.text if review_title_element else None
    
                # Extracting the review content
                review_content_element = li_tag.select_one('span.tl-m.db-m')
                product['Review Content'] = review_content_element.text.strip() if review_content_element else None
    
                # Extracting the reviewer's name
                review_name_element = li_tag.select_one('.f7.b.mv0')
                product['Review name'] = review_name_element.text if review_name_element else None
    
                # Extracting syndicated source,
                syndication_element = li_tag.select_one('.flex.f7 span.gray')
                if syndication_element and 'Review from' in syndication_element.text:
                    product['Syndicated source'] = syndication_element.text.split('Review from ')[-1].strip()
                else:
                    product['Syndicated source'] = None  # Assign None if no syndicated source is found
    
                # Correctly specify the button's aria-label as it appears in your HTML snippet
                helpful_element = soup.select_one('button[aria-label^="Upvote ndmomma review"] span')
                
                # Extract the number of people who found the review helpful
                people_find_helpful = int(helpful_element.text.strip('()')) if helpful_element else 0
    
                # Adding the URL of the review
                product['URL'] = url
    
                # Append the extracted product information to the list of reviews
                extracted_reviews.append(product)

    except Exception as e:
        print(f"Error encountered: {e}. Retrying in 5 seconds...")
        time.sleep(5)

    return extracted_reviews


# %%
urls = [
    'https://www.walmart.com/reviews/product/5129928603'
    # 'https://www.walmart.com/reviews/product/5129928603'
]

# %%

walmart_reviews = []

for link in urls:
    # initial value don't modify
    retry_count = 0

    # you can modify with your need
    max_try = 5
    retry_limit = max_try
    print(link)
    while retry_count < max_try:
        try:
            last_page_number = get_page_number(link)
            if last_page_number is None:
                retry_count += 1
                if retry_count <= retry_limit:
                    print("Failed to retrieve last page number. Retrying... Also Extract the data")
                    if retry_count == 1:
                        for page_number in range(1, last_page_number + 1):
                            retry_count = 0  # Reset retry count for each page
                            while retry_count < max_try:
                                try:
                                    target_url = f'{link}?page={page_number}'
                                    extracted_reviews = get_review_walmart(target_url)

                                    if len(extracted_reviews) == 0:
                                        print('No reviews found. Retrying in 5 seconds...')
                                        retry_count += 1
                                        time.sleep(5)
                                    else:
                                        walmart_reviews.extend(extracted_reviews)
                                        print(f'Review count in page {page_number}:', len(extracted_reviews))
                                        time.sleep(2)
                                        break

                                except Exception as e:
                                    print(f"Error encountered: {e}. Retrying in 3 seconds...")
                                    retry_count += 1
                                    time.sleep(3)
                            else:
                                print(f"Max retries exceeded for page {page_number}. Skipping to the next page.")

                    time.sleep(3)
                    continue
                else:
                    print("Failed to retrieve last page number after multiple retries. Changing the link.")
                    # Change the link here
                    link = new_link
                    retry_count = 0  # Reset retry count
                    continue
            print('Total pages:', last_page_number)
            break
        except Exception as e:
            print(f"Error encountered: {e}. Retrying in 3 seconds...")
            time.sleep(3)
        retry_count += 1
    else:
        print("Max retries exceeded for this link. Moving to the next link.")
        continue  # Move to the next link if max retries exceeded

    if last_page_number is None:
        print("Skipping processing for this link due to inability to retrieve last page number.")
        continue  # Move to the next link if last_page_number is None

    for page_number in range(1, last_page_number + 1):
        retry_count = 0  # Reset retry count for each page
        while retry_count < max_try:
            try:
                target_url = f'{link}?page={page_number}'
                extracted_reviews = get_review_walmart(target_url)

                if len(extracted_reviews) == 0:
                    print('No reviews found. Retrying in 5 seconds...')
                    retry_count += 1
                    time.sleep(5)
                else:
                    walmart_reviews.extend(extracted_reviews)
                    print(f'Review count in page {page_number}:', len(extracted_reviews))
                    time.sleep(2)
                    break

            except Exception as e:
                print(f"Error encountered: {e}. Retrying in 3 seconds...")
                retry_count += 1
                time.sleep(3)
        else:
            print(f"Max retries exceeded for page {page_number}. Skipping to the next page.")



# %%
walmart = pd.DataFrame(walmart_reviews)
walmart['Retailer'] = "Walmart"

walmart['scraping_date'] = date.today().strftime('%Y/%m/%d')
walmart['scraping_date'] = pd.to_datetime(walmart['scraping_date']).dt.date
walmart['Review date'] = pd.to_datetime(walmart['Review date']).dt.date
walmart['Review rating'] = walmart['Review rating'].astype(str).str.replace(' out of 5 stars review', '').astype(int)
walmart.drop_duplicates(inplace=True)

walmart['HP Model Number'] = walmart['Model'].str.extract(r'(\d+e?)')

walmart['Review date'] = pd.to_datetime(walmart['Review date'])

walmart_hp_combine = pd.merge(walmart, df_amazon, on="HP Model Number", how="left")
walmart_hp_combine['Review Model'] = walmart_hp_combine['HP Model']

columns_to_drop = ['Model', 'HP Model Number', 'Comp Model number', 'HP Model']
walmart_hp_combine = walmart_hp_combine.drop(columns_to_drop, axis=1)

walmart_hp_combine = walmart_hp_combine.drop_duplicates()
walmart_hp_combine['Review Model'] = walmart_hp_combine['Review Model'].fillna("")
walmart_hp_combine['Competitor_Flag'] = walmart_hp_combine['Review Model'].apply(
    lambda x: 'No' if 'HP' in x else 'Yes'
)

walmart_hp_combine['Country'] = 'US'

column_mapping = {
    'Review date': 'Review_Date',
    'review_text': 'Review_Content',
    'Review rating': 'Review_Rating',
    'url': 'URL',
    'review_title': 'Review_Title',
    'Verified Purchase or not': 'Verified_Purchase_Flag',
    'reviewer_name': 'Review_Name',
    'syndication': 'Syndicated_Source',
    'stars': 'Review_Rating',
    'Retailer': 'Retailer',
    'scraping_date': 'Scraping_Date',
    'Comp Model': 'Comp_Model',
    'HP Class': 'HP_Class',
    'Review Model': 'Review_Model',
    'Review title': 'Review_Title',
    'Review Content': 'Review_Content',
    'Review date': 'Review_Date',
    'URL': 'URL',
    'Seeding or not': 'Seeding_Flag',
    'Review name': 'Review_Name',
    'People_find_helpful': 'People_Find_Helpful',
    'Syndicated source': 'Syndicated_Source',
    'Comp Model': 'Comp_Model',
    'HP Class': 'HP_Class',
    'Review Model': 'Review_Model',
    'Competitor_Flag': 'Competitor_Flag'
}

# Rename columns in the original DataFrame
walmart_hp_combine = walmart_hp_combine.rename(columns=column_mapping)

# Concatenate with an empty DataFrame
Final_review = pd.concat([pd.DataFrame(), walmart_hp_combine], ignore_index=True)

# Add default values for some columns
Final_review['Country'] = 'US'
Final_review['Review_Date'] = pd.to_datetime(Final_review['Review_Date']).dt.date
Final_review['Review_Rating'] = Final_review['Review_Rating'].astype('int64', errors='ignore')
Final_review['Review_Rating_Label'] = Final_review['Review_Rating'].apply(lambda x: '1-2-3-star' if x <4 else '4-5-star') 
# Handle missing 'People_Find_Helpful' column
if 'People_Find_Helpful' in Final_review.columns:
    Final_review['People_Find_Helpful'] = Final_review['People_Find_Helpful'].fillna(0).astype('int64')
else:
    Final_review['People_Find_Helpful'] = 0

Final_review['Scraping_Date'] = pd.to_datetime(Final_review['Scraping_Date']).dt.date

# Fill NaN values in string columns with empty string
string_columns = Final_review.select_dtypes(include='object').columns
Final_review[string_columns] = Final_review[string_columns].fillna('')

# Ensure all required columns are present
required_columns = [
    'Review_Model', 'Competitor_Flag', 'HP_Class', 'Segment', 'Retailer',
    'Comp_Model', 'Review_Date', 'Review_Name', 'Review_Rating',
    'Review_Rating_Label', 'Review_Title', 'Review_Content', 'Seeding_Flag',
    'Verified_Purchase_Flag', 'Promotion_Flag', 'Aggregation_Flag',
    'People_Find_Helpful', 'Syndicated_Source', 'Response_Date',
    'Response_Text', 'Response_Name', 'URL', 'Scraping_Date', 'Country',
    'Orginal_Title', 'Orginal Title'
]

for col in required_columns:
    if col not in Final_review.columns:
        Final_review[col] = None

# Reorder columns to match the required_columns list
Final_review = Final_review[required_columns]

# change here
previous = pd.read_csv(path +'Tassel_EMEA_Review_Raw.csv')
# previous['Review_Date'] = pd.to_datetime(previous['Review_Date'], format='mixed').dt.date
# previous['Scraping_Date'] = pd.to_datetime(previous['Scraping_Date'], format='mixed').dt.date
previous['Review_Date'] = pd.to_datetime(previous['Review_Date'], errors='coerce').dt.date
previous['Scraping_Date'] = pd.to_datetime(previous['Scraping_Date'], errors='coerce').dt.date
previous['Review_Rating'] = previous['Review_Rating'].astype(int)



# Save to CSV
Final_review.to_csv('walmart.csv', index=False)
print('Walmart Script running completed. Walmart.csv file saved')